#### 1. Bibliotecas


In [2]:
import pandas as pd
import datetime

#### 2. Funções

In [18]:
def aquisicao_dados(local):

    # Leitura do extrato
    df = pd.read_csv(local)

    # Print do extrato
    print('*'*100, '\n', f'Extrato: {local}', '\n', '*'*100)
    print(df)

    return df


In [19]:
def filtro_primario(dados, tipo):

    if tipo == 'Entrada':
        # Filtro primario
        df_filtrado = dados[dados['Valor'] > 0]

        # Print do extrato
        print('*'*100, '\n', f'Extrato {tipo}', '\n', '*'*100)
        print(df_filtrado)

    if tipo == 'Saida':
        # Filtro primario
        df_filtrado = dados[dados['Valor'] < 0]

        # Print do extrato
        print('*'*100, '\n', f'Extrato {tipo}', '\n', '*'*100)
        print(df_filtrado)

    return df_filtrado
        

In [20]:
def troca_formato_data(dados):

    dados['Data'] = dados['Data'].apply(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').strftime('%Y-%m-%d'))

    return dados

In [23]:
def tratamento_dados(dados):

    # Troca do formato da data
    dados = troca_formato_data(dados)

    # Renomeação das colunas 'Data' e 'Valor'
    dados.rename(columns = {'Data': 'data_pagamento'}, inplace = True)
    dados.rename(columns = {'Valor': 'valor'}, inplace = True)

    # Split da 'Descrição' em 4 colunas
    dados['categoria'] = dados['Descrição'].str.split(' - ', expand=True)[0]
    dados['origem'] = dados['Descrição'].str.split(' - ', expand=True)[1]
    dados['cpf'] = dados['Descrição'].str.split(' - ', expand=True)[2]
    dados['dados_bancarios'] = dados['Descrição'].str.split(' - ', expand=True)[3]

    # Retirada da coluna 'Descrição'
    dados = dados.drop('Descrição', axis=1)

    # Print do extrato
    print('*'*100, '\n', f'Extrato com dados tratados', '\n', '*'*100)
    print(dados)

    return dados


In [27]:
def operacao_extracao(local, tipo):
    
    df = aquisicao_dados(local)

    df_filtrado = filtro_primario(df, tipo)

    df_tratado = tratamento_dados(df_filtrado)

    return df_tratado

In [32]:
def print_MySQL(dados, ultimo_id):

    in_string = 'INSERT INTO `DADOS_BANCO` (`id_BAN`, `BAN_ORIGEM`, `BAN_CATEGORIA`, `BAN_VALOR`, `BAN_DATA`) VALUES'

    print(in_string)

    for i in df.index:

        origem = dados.loc[i, 'origem']
        categoria = dados.loc[i, 'categoria']
        data_pagamento = dados.loc[i, 'data_pagamento']
        valor = dados.loc[i, 'valor']

        ultimo_id += 1

        string = (ultimo_id, f'{origem}', f'{categoria}', valor, f'{data_pagamento}')

        print(string)

#### 3. Aplicação

Para receber o extrato do seu NuBank, originalmente em .csv, em uma tabela tratada e resumida, pronta para ser adequada para o armazenamento em SQL, você precisara do *path* do arquivo .csv 

Ademais, também será necessário especicificar se deseja obter os valores de entrada ou saída do extrato

In [ ]:
dados = operacao_extracao('path_do_arquivo_aqui','Entrada ou Saida')

Para visualizar a tabela, apenas digite 'dados' ao compilador

In [ ]:
dados

Caso queira realizar operações especificar, aqui vai uma lista de possíveis operações:

1. Localização de um dado específico

In [ ]:
dados['nome_da_coluna'].loc['id']

2. Soma dos valores
   

In [ ]:
sum(dados['valor'])

3. Retirada de uma linha específica da base de dados

In [ ]:
dados = dados.drop(index='id')

4. Adição de um dado específico

In [ ]:
dados.loc['id', 'nome_da_coluna'] = 'informacao'

#### 4. Adequação dos dados para a linguagem SQL

O Banco de Dados em MySQL possui a estrutura explicada em: "script_banco_dados_entrada.sql"

In [ ]:
print_MySQL(dados, ultimo_id)